In [ ]:
baseline3

In [ ]:
直接使用baseline3 split获得的dataset来进行xgb train

In [1]:
import numpy as np
import pandas as pd

In [2]:
t831913 = pd.read_csv('dataset/final/t831913.csv')
test914 = pd.read_csv('dataset/final/test914.csv')
t901914 = pd.read_csv('dataset/final/t901914.csv')
test_A = pd.read_csv('dataset/final/test_A.csv')
test_B = pd.read_csv('dataset/final/test_B.csv')

In [3]:
y831913 = pd.read_csv('dataset/final/y831913.csv',header = None)
y901914 = pd.read_csv('dataset/final/y901914.csv',header = None)
y914 = pd.read_csv('dataset/final/y914.csv',header = None)

In [116]:
t831913.shape,t901914.shape,test914.shape,test_A.shape,test_B.shape

((307564, 21), (309802, 21), (23846, 21), (11035, 21), (11036, 21))

In [117]:
y831913.shape,y901914.shape,y914.shape

((307564, 1), (309802, 1), (23846, 1))

In [118]:
t831913.columns

Index(['city', 'countryid', 'glat', 'glon', 'hotelbelongto', 'hotelstar',
       'isebookinghtl', 'isholdroom', 'isvendor', 'ordadvanceday', 'orderid',
       'room', 'supplierchannel', 'supplierid', 'wuli_totalrooms',
       'zifangxing_totalrooms', 'zone', 'order_hour', 'order_minute',
       'day_stay', 'room_order_number'],
      dtype='object')

In [ ]:
必须调整test的列顺序，保持和train一致才行。不然算法会报错。

In [4]:
cols = list(t831913.columns)
test_A = test_A.loc[:,cols]
test_B = test_B.loc[:,cols]

In [5]:
y831913.columns = ['noroom']
y901914.columns = ['noroom']
y914.columns = ['noroom']

y831913 = pd.Series(y831913['noroom'])
y901914 = pd.Series(y901914['noroom'])
y914 = pd.Series(y914['noroom'])

In [ ]:
使用到的辅助def

In [6]:
from sklearn.metrics import precision_recall_curve

In [ ]:
pr曲线计算线下面积

In [7]:
def pr(y_true,y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true,y_pred)
    precision = pd.Series(precision)
    recall = pd.Series(recall)
    recall = recall[recall >= 0.05]
    recall = recall[recall <= 0.50]
    precision = precision[recall.index]
    x = (0.50-0.05) / recall.shape[0]
    sum = 0
    for item in precision:
        sum = sum + item * x
    
    return sum,precision,recall    

In [ ]:
count01的个数

In [8]:
def pred(y_pred,threshold):
    pred01 = []
    for item in y_pred:
        if item > threshold:
            pred01.append(1)
        else:
            pred01.append(0)
    pred01 = pd.Series(pred01)
    return pred01

In [124]:
t831913.columns

Index(['city', 'countryid', 'glat', 'glon', 'hotelbelongto', 'hotelstar',
       'isebookinghtl', 'isholdroom', 'isvendor', 'ordadvanceday', 'orderid',
       'room', 'supplierchannel', 'supplierid', 'wuli_totalrooms',
       'zifangxing_totalrooms', 'zone', 'order_hour', 'order_minute',
       'day_stay', 'room_order_number'],
      dtype='object')

In [ ]:
718变更：
将一部分老特征加回来了：
id类的hotel酒店号，masterbasicroomid物理房型号, masterhotelid母酒店号

In [ ]:
经纬度观察
——结论是越复杂结果越好。因此不要做round的操作了。

In [ ]:
尝试使用lgb

In [170]:
import lightgbm as lgb

In [171]:
drop_list = ['orderid','room_order_number']

In [178]:
t831913.columns

Index(['city', 'countryid', 'glat', 'glon', 'hotelbelongto', 'hotelstar',
       'isebookinghtl', 'isholdroom', 'isvendor', 'ordadvanceday', 'orderid',
       'room', 'supplierchannel', 'supplierid', 'wuli_totalrooms',
       'zifangxing_totalrooms', 'zone', 'order_hour', 'order_minute',
       'day_stay', 'room_order_number'],
      dtype='object')

In [179]:
def get_dataset(train_x,train_y):
    list_name = list(train_x.columns)
    list_name.pop(10) # 删掉orderid
    list_name.pop(19) # 删掉room_order_number
    train_x = train_x.drop(drop_list,axis = 1).values
    train_y = train_y.values
    lgb_train = lgb.Dataset(train_x, train_y)
    return list_name,lgb_train

In [180]:
list_name,lgb_train = get_dataset(t831913,y831913)

In [175]:
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',# 也可以尝试dart，goss
    'objective': 'binary',
    'metric': 'binary_logloss', # 不同的metric感觉就是最后指标计算不同，迭代方向是没有不同的
    'num_leaves': 255, # 31，255相当于8层
    'max_bin':255,
    'min_data_in_leaf':20,
    'learning_rate': 0.05,
    'feature_fraction': 0.9, # 0.9
    'bagging_fraction': 0.8,
    'bagging_freq': 5, # 每k次迭代，执行一次bagging
    'min_split_gain': 0, # default = 0
    'lambda_l1':0,
    'lambda_l2':0,
    'verbose': 0,
    'random_state':100
}

In [176]:
print('Start training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_train)  # eval training data
                #feature_name=feature_name)
                # categorical_feature=[21])

Start training...
[1]	training's binary_logloss: 0.651235
[2]	training's binary_logloss: 0.613244
[3]	training's binary_logloss: 0.578682
[4]	training's binary_logloss: 0.547148
[5]	training's binary_logloss: 0.51833
[6]	training's binary_logloss: 0.491862
[7]	training's binary_logloss: 0.467487
[8]	training's binary_logloss: 0.44499
[9]	training's binary_logloss: 0.424201
[10]	training's binary_logloss: 0.404966
[11]	training's binary_logloss: 0.387113
[12]	training's binary_logloss: 0.370521
[13]	training's binary_logloss: 0.355092
[14]	training's binary_logloss: 0.340742
[15]	training's binary_logloss: 0.32736
[16]	training's binary_logloss: 0.314834
[17]	training's binary_logloss: 0.303137
[18]	training's binary_logloss: 0.292441
[19]	training's binary_logloss: 0.282217
[20]	training's binary_logloss: 0.27262
[21]	training's binary_logloss: 0.263663
[22]	training's binary_logloss: 0.255309
[23]	training's binary_logloss: 0.247423
[24]	training's binary_logloss: 0.240069
[25]	traini

[200]	training's binary_logloss: 0.0883561
[201]	training's binary_logloss: 0.0881456
[202]	training's binary_logloss: 0.08798
[203]	training's binary_logloss: 0.0878074
[204]	training's binary_logloss: 0.0876436
[205]	training's binary_logloss: 0.0874558
[206]	training's binary_logloss: 0.0872777
[207]	training's binary_logloss: 0.0870848
[208]	training's binary_logloss: 0.0869174
[209]	training's binary_logloss: 0.086747
[210]	training's binary_logloss: 0.0866001
[211]	training's binary_logloss: 0.0864439
[212]	training's binary_logloss: 0.0862844
[213]	training's binary_logloss: 0.0861496
[214]	training's binary_logloss: 0.0859995
[215]	training's binary_logloss: 0.0858588
[216]	training's binary_logloss: 0.0857078
[217]	training's binary_logloss: 0.0855643
[218]	training's binary_logloss: 0.0853775
[219]	training's binary_logloss: 0.0852499
[220]	training's binary_logloss: 0.0850536
[221]	training's binary_logloss: 0.0848956
[222]	training's binary_logloss: 0.0847195
[223]	training

[399]	training's binary_logloss: 0.0623497
[400]	training's binary_logloss: 0.0622344
[401]	training's binary_logloss: 0.0621324
[402]	training's binary_logloss: 0.062051
[403]	training's binary_logloss: 0.0619625
[404]	training's binary_logloss: 0.0618721
[405]	training's binary_logloss: 0.0617891
[406]	training's binary_logloss: 0.0616766
[407]	training's binary_logloss: 0.0615715
[408]	training's binary_logloss: 0.0614761
[409]	training's binary_logloss: 0.0613815
[410]	training's binary_logloss: 0.0613028
[411]	training's binary_logloss: 0.0612349
[412]	training's binary_logloss: 0.0611598
[413]	training's binary_logloss: 0.0610966
[414]	training's binary_logloss: 0.0610296
[415]	training's binary_logloss: 0.0609612
[416]	training's binary_logloss: 0.0608335
[417]	training's binary_logloss: 0.0607248
[418]	training's binary_logloss: 0.0606324
[419]	training's binary_logloss: 0.0605181
[420]	training's binary_logloss: 0.0604116
[421]	training's binary_logloss: 0.0603246
[422]	traini

[591]	training's binary_logloss: 0.0461132
[592]	training's binary_logloss: 0.0460538
[593]	training's binary_logloss: 0.0459795
[594]	training's binary_logloss: 0.0459233
[595]	training's binary_logloss: 0.0458696
[596]	training's binary_logloss: 0.0457885
[597]	training's binary_logloss: 0.0457091
[598]	training's binary_logloss: 0.0456316
[599]	training's binary_logloss: 0.0455556
[600]	training's binary_logloss: 0.0454771
[601]	training's binary_logloss: 0.0454086
[602]	training's binary_logloss: 0.0453437
[603]	training's binary_logloss: 0.0452711
[604]	training's binary_logloss: 0.0452002
[605]	training's binary_logloss: 0.0451271
[606]	training's binary_logloss: 0.0450522
[607]	training's binary_logloss: 0.0449849
[608]	training's binary_logloss: 0.0449111
[609]	training's binary_logloss: 0.0448335
[610]	training's binary_logloss: 0.0447689
[611]	training's binary_logloss: 0.0446922
[612]	training's binary_logloss: 0.0446239
[613]	training's binary_logloss: 0.0445565
[614]	train

[784]	training's binary_logloss: 0.0348246
[785]	training's binary_logloss: 0.034771
[786]	training's binary_logloss: 0.0347156
[787]	training's binary_logloss: 0.0346802
[788]	training's binary_logloss: 0.0346367
[789]	training's binary_logloss: 0.0346045
[790]	training's binary_logloss: 0.0345536
[791]	training's binary_logloss: 0.0345122
[792]	training's binary_logloss: 0.0344698
[793]	training's binary_logloss: 0.0344221
[794]	training's binary_logloss: 0.0343757
[795]	training's binary_logloss: 0.0343339
[796]	training's binary_logloss: 0.034285
[797]	training's binary_logloss: 0.0342343
[798]	training's binary_logloss: 0.0341843
[799]	training's binary_logloss: 0.0341395
[800]	training's binary_logloss: 0.034101
[801]	training's binary_logloss: 0.0340429
[802]	training's binary_logloss: 0.033981
[803]	training's binary_logloss: 0.0339251
[804]	training's binary_logloss: 0.0338696
[805]	training's binary_logloss: 0.0338065
[806]	training's binary_logloss: 0.0337483
[807]	training'

...

[2111]	training's binary_logloss: 0.00608331
[2112]	training's binary_logloss: 0.00607548
[2113]	training's binary_logloss: 0.0060682
[2114]	training's binary_logloss: 0.00605962
[2115]	training's binary_logloss: 0.00605226
[2116]	training's binary_logloss: 0.00604471
[2117]	training's binary_logloss: 0.00603786
[2118]	training's binary_logloss: 0.00603019
[2119]	training's binary_logloss: 0.00602367
[2120]	training's binary_logloss: 0.00601714
[2121]	training's binary_logloss: 0.00601085
[2122]	training's binary_logloss: 0.00600315
[2123]	training's binary_logloss: 0.00599504
[2124]	training's binary_logloss: 0.00598811
[2125]	training's binary_logloss: 0.0059824
[2126]	training's binary_logloss: 0.00597584
[2127]	training's binary_logloss: 0.00596876
[2128]	training's binary_logloss: 0.00596014
[2129]	training's binary_logloss: 0.00595366
[2130]	training's binary_logloss: 0.00594796
[2131]	training's binary_logloss: 0.00594054
[2132]	training's binary_logloss: 0.00593309
[2133]	train

[2294]	training's binary_logloss: 0.00487562
[2295]	training's binary_logloss: 0.00486919
[2296]	training's binary_logloss: 0.00486477
[2297]	training's binary_logloss: 0.0048595
[2298]	training's binary_logloss: 0.00485537
[2299]	training's binary_logloss: 0.00485144
[2300]	training's binary_logloss: 0.00484732
[2301]	training's binary_logloss: 0.00484035
[2302]	training's binary_logloss: 0.004835
[2303]	training's binary_logloss: 0.00482996
[2304]	training's binary_logloss: 0.00482479
[2305]	training's binary_logloss: 0.00481935
[2306]	training's binary_logloss: 0.00481267
[2307]	training's binary_logloss: 0.00480709
[2308]	training's binary_logloss: 0.00480067
[2309]	training's binary_logloss: 0.00479421
[2310]	training's binary_logloss: 0.00478861
[2311]	training's binary_logloss: 0.00478374
[2312]	training's binary_logloss: 0.00477882
[2313]	training's binary_logloss: 0.00477317
[2314]	training's binary_logloss: 0.00476882
[2315]	training's binary_logloss: 0.00476367
[2316]	traini

[2478]	training's binary_logloss: 0.00397692
[2479]	training's binary_logloss: 0.00397273
[2480]	training's binary_logloss: 0.00396811
[2481]	training's binary_logloss: 0.00396382
[2482]	training's binary_logloss: 0.0039602
[2483]	training's binary_logloss: 0.00395667
[2484]	training's binary_logloss: 0.00395289
[2485]	training's binary_logloss: 0.00394933
[2486]	training's binary_logloss: 0.0039451
[2487]	training's binary_logloss: 0.00394042
[2488]	training's binary_logloss: 0.003937
[2489]	training's binary_logloss: 0.00393337
[2490]	training's binary_logloss: 0.00393001
[2491]	training's binary_logloss: 0.00392522
[2492]	training's binary_logloss: 0.00392043
[2493]	training's binary_logloss: 0.00391606
[2494]	training's binary_logloss: 0.003911
[2495]	training's binary_logloss: 0.00390763
[2496]	training's binary_logloss: 0.00390236
[2497]	training's binary_logloss: 0.00389748
[2498]	training's binary_logloss: 0.00389286
[2499]	training's binary_logloss: 0.00388884
[2500]	training'

[2666]	training's binary_logloss: 0.00323775
[2667]	training's binary_logloss: 0.00323354
[2668]	training's binary_logloss: 0.00322977
[2669]	training's binary_logloss: 0.00322678
[2670]	training's binary_logloss: 0.00322413
[2671]	training's binary_logloss: 0.00322061
[2672]	training's binary_logloss: 0.00321746
[2673]	training's binary_logloss: 0.00321513
[2674]	training's binary_logloss: 0.00321254
[2675]	training's binary_logloss: 0.00321023
[2676]	training's binary_logloss: 0.00320621
[2677]	training's binary_logloss: 0.00320323
[2678]	training's binary_logloss: 0.00320036
[2679]	training's binary_logloss: 0.00319811
[2680]	training's binary_logloss: 0.00319581
[2681]	training's binary_logloss: 0.0031921
[2682]	training's binary_logloss: 0.00318924
[2683]	training's binary_logloss: 0.00318565
[2684]	training's binary_logloss: 0.00318211
[2685]	training's binary_logloss: 0.00317923
[2686]	training's binary_logloss: 0.00317522
[2687]	training's binary_logloss: 0.00317176
[2688]	trai

[2849]	training's binary_logloss: 0.00267517
[2850]	training's binary_logloss: 0.00267308
[2851]	training's binary_logloss: 0.00266986
[2852]	training's binary_logloss: 0.00266702
[2853]	training's binary_logloss: 0.00266411
[2854]	training's binary_logloss: 0.00266165
[2855]	training's binary_logloss: 0.00265986
[2856]	training's binary_logloss: 0.00265669
[2857]	training's binary_logloss: 0.00265357
[2858]	training's binary_logloss: 0.00265171
[2859]	training's binary_logloss: 0.0026498
[2860]	training's binary_logloss: 0.0026486
[2861]	training's binary_logloss: 0.00264516
[2862]	training's binary_logloss: 0.00264241
[2863]	training's binary_logloss: 0.00264021
[2864]	training's binary_logloss: 0.00263786
[2865]	training's binary_logloss: 0.00263622
[2866]	training's binary_logloss: 0.00263306
[2867]	training's binary_logloss: 0.00263021
[2868]	training's binary_logloss: 0.00262769
[2869]	training's binary_logloss: 0.00262528
[2870]	training's binary_logloss: 0.00262298
[2871]	train

In [181]:
feat_importance = pd.Series(index = list_name,data = gbm.feature_importance())
feat_importance.sort_values(ascending = False)

order_minute             111600
wuli_totalrooms           83739
ordadvanceday             81035
room                      78854
zifangxing_totalrooms     70453
order_hour                64464
glat                      60225
city                      49052
supplierid                39766
zone                      32517
day_stay                  26676
countryid                 22659
hotelstar                 16717
glon                       6223
supplierchannel            5146
isebookinghtl              3807
hotelbelongto              3478
isvendor                   2926
isholdroom                 2663
dtype: int32

In [182]:
y_pred914 = gbm.predict(test914)

In [183]:
y_pred914

array([  4.49015119e-07,   8.10630927e-04,   1.96917165e-05, ...,
         1.70022763e-07,   4.96598667e-08,   1.53238446e-05])

In [184]:
pr_sum,precision,recall = pr(y914, y_pred914)

In [185]:
pr_sum

0.044141704442373586

In [ ]:
y_pred = gbm.predict(test_A.drop(drop_list,axis = 1))

In [ ]:
pred01 = pred(y_pred,0.5)
pred01.value_counts()

In [ ]:
提交可以用下边xgb的

In [ ]:
使用xgb

In [9]:
import xgboost as xgb

E:\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
先观察order_second的单独效果

In [10]:
drop_list = ['orderid','room_order_number',
          'hotel','masterbasicroomid', 'masterhotelid']

In [66]:
drop_list = ['orderid','room_order_number',
          'hotel','masterbasicroomid'] # 加上'masterhotelid'

In [40]:
drop_list = ['orderid',
          'hotel','masterbasicroomid'] # 加上'masterhotelid'，加上number

In [12]:
drop_list = ['orderid','room_order_number']
# room_order_number没处理好之前，是不用了。

In [60]:
drop_list = ['orderid']

In [13]:
# 这一组用于使用914来验证结果，因为915给不出结果来
dataset3_0 = xgb.DMatrix(t831913.drop(drop_list,axis = 1),label = y831913)
dataset4_0 = xgb.DMatrix(test914.drop(drop_list,axis = 1),label = y914)
dataset3 = xgb.DMatrix(t901914.drop(drop_list,axis = 1),label = y901914)
dataset4 = xgb.DMatrix(test_A.drop(drop_list,axis = 1))
dataset5 = xgb.DMatrix(test_B.drop(drop_list,axis = 1))

In [14]:
params={
'booster':'gbtree',
'objective':'binary:logistic',
# 'eval_metric': 'auc', # 评价标准是f1，因此需要自己写feval
'tree_method':'exact', # 对于小数据集，xgb使用原版（贪心算法），不需要使用近似算法
'gamma':0.1,
'max_depth':9, # 7
'lambda':0, # 10
'subsample':0.95, # 0.7
'colsample_bytree':0.95, # 0.7
'min_child_weight':1,  # 1.1
'colsample_bylevel':0.95, # 0.7
'eta': 0.01, 
'seed':1000,
'eval_metric': 'logloss' # 试试loss吧
}

In [141]:
# model3
watchlist = [(dataset3_0,'train')]
model_3_0 = xgb.train(params,dataset3_0,num_boost_round = 2000,evals = watchlist,
                    early_stopping_rounds = 100)

feature_score_3_0 = model_3_0.get_fscore()
feature_score_3_0 = sorted(feature_score_3_0.items(), key=lambda x:x[1],reverse=True)

[0]	train-logloss:0.684616
Will train until train-logloss hasn't improved in 100 rounds.
[1]	train-logloss:0.676254
[2]	train-logloss:0.668048
[3]	train-logloss:0.660008
[4]	train-logloss:0.652117
[5]	train-logloss:0.644436
[6]	train-logloss:0.636845
[7]	train-logloss:0.629386
[8]	train-logloss:0.622073
[9]	train-logloss:0.614895
[10]	train-logloss:0.607852
[11]	train-logloss:0.600941
[12]	train-logloss:0.594141
[13]	train-logloss:0.587475
[14]	train-logloss:0.580921
[15]	train-logloss:0.57449
[16]	train-logloss:0.568212
[17]	train-logloss:0.561991
[18]	train-logloss:0.555934
[19]	train-logloss:0.549934
[20]	train-logloss:0.544042
[21]	train-logloss:0.53824
[22]	train-logloss:0.532532
[23]	train-logloss:0.526923
[24]	train-logloss:0.521406
[25]	train-logloss:0.515985
[26]	train-logloss:0.510642
[27]	train-logloss:0.505394
[28]	train-logloss:0.500233
[29]	train-logloss:0.495206
[30]	train-logloss:0.490215
[31]	train-logloss:0.485304
[32]	train-logloss:0.480464
[33]	train-logloss:0.47571

[286]	train-logloss:0.139603
[287]	train-logloss:0.139388
[288]	train-logloss:0.139166
[289]	train-logloss:0.138957
[290]	train-logloss:0.138736
[291]	train-logloss:0.138531
[292]	train-logloss:0.138329
[293]	train-logloss:0.13813
[294]	train-logloss:0.137916
[295]	train-logloss:0.137717
[296]	train-logloss:0.13751
[297]	train-logloss:0.137321
[298]	train-logloss:0.137126
[299]	train-logloss:0.136937
[300]	train-logloss:0.136746
[301]	train-logloss:0.13656
[302]	train-logloss:0.13638
[303]	train-logloss:0.136203
[304]	train-logloss:0.136013
[305]	train-logloss:0.135817
[306]	train-logloss:0.135637
[307]	train-logloss:0.135462
[308]	train-logloss:0.135282
[309]	train-logloss:0.135115
[310]	train-logloss:0.134952
[311]	train-logloss:0.134782
[312]	train-logloss:0.134622
[313]	train-logloss:0.134451
[314]	train-logloss:0.134285
[315]	train-logloss:0.134125
[316]	train-logloss:0.133969
[317]	train-logloss:0.133809
[318]	train-logloss:0.133648
[319]	train-logloss:0.133493
[320]	train-loglos

[570]	train-logloss:0.11467
[571]	train-logloss:0.114632
[572]	train-logloss:0.1146
[573]	train-logloss:0.114566
[574]	train-logloss:0.11454
[575]	train-logloss:0.114496
[576]	train-logloss:0.114466
[577]	train-logloss:0.114437
[578]	train-logloss:0.114402
[579]	train-logloss:0.114372
[580]	train-logloss:0.114344
[581]	train-logloss:0.114311
[582]	train-logloss:0.114274
[583]	train-logloss:0.114227
[584]	train-logloss:0.11419
[585]	train-logloss:0.114164
[586]	train-logloss:0.114137
[587]	train-logloss:0.114103
[588]	train-logloss:0.114084
[589]	train-logloss:0.114053
[590]	train-logloss:0.114021
[591]	train-logloss:0.113988
[592]	train-logloss:0.113951
[593]	train-logloss:0.113909
[594]	train-logloss:0.113878
[595]	train-logloss:0.113851
[596]	train-logloss:0.113806
[597]	train-logloss:0.113774
[598]	train-logloss:0.11375
[599]	train-logloss:0.113717
[600]	train-logloss:0.113697
[601]	train-logloss:0.113659
[602]	train-logloss:0.113629
[603]	train-logloss:0.113603
[604]	train-logloss:

[854]	train-logloss:0.108088
[855]	train-logloss:0.108076
[856]	train-logloss:0.108059
[857]	train-logloss:0.10804
[858]	train-logloss:0.108019
[859]	train-logloss:0.108002
[860]	train-logloss:0.107973
[861]	train-logloss:0.10795
[862]	train-logloss:0.107924
[863]	train-logloss:0.107914
[864]	train-logloss:0.107898
[865]	train-logloss:0.107888
[866]	train-logloss:0.107873
[867]	train-logloss:0.107847
[868]	train-logloss:0.107827
[869]	train-logloss:0.107817
[870]	train-logloss:0.107814
[871]	train-logloss:0.107786
[872]	train-logloss:0.107761
[873]	train-logloss:0.107745
[874]	train-logloss:0.10772
[875]	train-logloss:0.107701
[876]	train-logloss:0.10767
[877]	train-logloss:0.107657
[878]	train-logloss:0.107636
[879]	train-logloss:0.107627
[880]	train-logloss:0.107609
[881]	train-logloss:0.107601
[882]	train-logloss:0.107584
[883]	train-logloss:0.10758
[884]	train-logloss:0.107574
[885]	train-logloss:0.107572
[886]	train-logloss:0.107563
[887]	train-logloss:0.107554
[888]	train-logloss

[1134]	train-logloss:0.103029
[1135]	train-logloss:0.103012
[1136]	train-logloss:0.102994
[1137]	train-logloss:0.102989
[1138]	train-logloss:0.102947
[1139]	train-logloss:0.102943
[1140]	train-logloss:0.102932
[1141]	train-logloss:0.102895
[1142]	train-logloss:0.102883
[1143]	train-logloss:0.102871
[1144]	train-logloss:0.102857
[1145]	train-logloss:0.102851
[1146]	train-logloss:0.102843
[1147]	train-logloss:0.102815
[1148]	train-logloss:0.102807
[1149]	train-logloss:0.102781
[1150]	train-logloss:0.102774
[1151]	train-logloss:0.102762
[1152]	train-logloss:0.102744
[1153]	train-logloss:0.102739
[1154]	train-logloss:0.102726
[1155]	train-logloss:0.102717
[1156]	train-logloss:0.102693
[1157]	train-logloss:0.102663
[1158]	train-logloss:0.102634
[1159]	train-logloss:0.10263
[1160]	train-logloss:0.102615
[1161]	train-logloss:0.1026
[1162]	train-logloss:0.102585
[1163]	train-logloss:0.102559
[1164]	train-logloss:0.102537
[1165]	train-logloss:0.102509
[1166]	train-logloss:0.102482
[1167]	train-

[1409]	train-logloss:0.098016
[1410]	train-logloss:0.097988
[1411]	train-logloss:0.097974
[1412]	train-logloss:0.097943
[1413]	train-logloss:0.097931
[1414]	train-logloss:0.09791
[1415]	train-logloss:0.097896
[1416]	train-logloss:0.097888
[1417]	train-logloss:0.097869
[1418]	train-logloss:0.097852
[1419]	train-logloss:0.097825
[1420]	train-logloss:0.097779
[1421]	train-logloss:0.097775
[1422]	train-logloss:0.097759
[1423]	train-logloss:0.097737
[1424]	train-logloss:0.097721
[1425]	train-logloss:0.097714
[1426]	train-logloss:0.097684
[1427]	train-logloss:0.097679
[1428]	train-logloss:0.097643
[1429]	train-logloss:0.097638
[1430]	train-logloss:0.097618
[1431]	train-logloss:0.097576
[1432]	train-logloss:0.097561
[1433]	train-logloss:0.097537
[1434]	train-logloss:0.097504
[1435]	train-logloss:0.097489
[1436]	train-logloss:0.097482
[1437]	train-logloss:0.097464
[1438]	train-logloss:0.097452
[1439]	train-logloss:0.097441
[1440]	train-logloss:0.097431
[1441]	train-logloss:0.097394
[1442]	trai

[1683]	train-logloss:0.093186
[1684]	train-logloss:0.09317
[1685]	train-logloss:0.093154
[1686]	train-logloss:0.093139
[1687]	train-logloss:0.093127
[1688]	train-logloss:0.093111
[1689]	train-logloss:0.093091
[1690]	train-logloss:0.093077
[1691]	train-logloss:0.093058
[1692]	train-logloss:0.093023
[1693]	train-logloss:0.093004
[1694]	train-logloss:0.09298
[1695]	train-logloss:0.092978
[1696]	train-logloss:0.092954
[1697]	train-logloss:0.092951
[1698]	train-logloss:0.092936
[1699]	train-logloss:0.092914
[1700]	train-logloss:0.092906
[1701]	train-logloss:0.09287
[1702]	train-logloss:0.092868
[1703]	train-logloss:0.092851
[1704]	train-logloss:0.09283
[1705]	train-logloss:0.092806
[1706]	train-logloss:0.0928
[1707]	train-logloss:0.092767
[1708]	train-logloss:0.092758
[1709]	train-logloss:0.092746
[1710]	train-logloss:0.092729
[1711]	train-logloss:0.09272
[1712]	train-logloss:0.092714
[1713]	train-logloss:0.092708
[1714]	train-logloss:0.092682
[1715]	train-logloss:0.09267
[1716]	train-loglo

[1957]	train-logloss:0.088858
[1958]	train-logloss:0.088827
[1959]	train-logloss:0.088807
[1960]	train-logloss:0.088804
[1961]	train-logloss:0.088791
[1962]	train-logloss:0.088773
[1963]	train-logloss:0.08877
[1964]	train-logloss:0.088768
[1965]	train-logloss:0.088762
[1966]	train-logloss:0.088745
[1967]	train-logloss:0.088735
[1968]	train-logloss:0.088716
[1969]	train-logloss:0.088695
[1970]	train-logloss:0.088683
[1971]	train-logloss:0.088666
[1972]	train-logloss:0.088658
[1973]	train-logloss:0.088637
[1974]	train-logloss:0.088622
[1975]	train-logloss:0.088616
[1976]	train-logloss:0.088612
[1977]	train-logloss:0.088584
[1978]	train-logloss:0.088575
[1979]	train-logloss:0.088553
[1980]	train-logloss:0.088548
[1981]	train-logloss:0.088542
[1982]	train-logloss:0.088527
[1983]	train-logloss:0.088506
[1984]	train-logloss:0.088483
[1985]	train-logloss:0.088468
[1986]	train-logloss:0.088454
[1987]	train-logloss:0.088435
[1988]	train-logloss:0.088434
[1989]	train-logloss:0.088427
[1990]	trai

In [97]:
y914.value_counts()

0.0    22595
1.0     1251
Name: noroom, dtype: int64

In [142]:
feature_score_3_0

[('room', 53211),
 ('wuli_totalrooms', 43849),
 ('glat', 37914),
 ('ordadvanceday', 36975),
 ('zifangxing_totalrooms', 35631),
 ('order_minute', 32950),
 ('order_hour', 25613),
 ('city', 25584),
 ('supplierid', 24741),
 ('zone', 20059),
 ('countryid', 13819),
 ('day_stay', 11706),
 ('hotelstar', 8655),
 ('glon', 4006),
 ('hotelbelongto', 2521),
 ('supplierchannel', 2041),
 ('isebookinghtl', 1719),
 ('isvendor', 1528),
 ('isholdroom', 1060)]

In [143]:
preds_3_0 = model_3_0.predict(dataset4_0)
pr_sum,precision,recall = pr(y914,preds_3_0)

In [26]:
一些取值范围
5 12000,13000 10000-14000
6 10000-13000 11000,12000都还行
7 4000-7000 （5000最好，4000,6000都可以，7000最差）
8 2000-5000 （3000最好，2000,4000次之，5000最差）

SyntaxError: invalid syntax (<ipython-input-26-4c2dbef3ee8c>, line 2)

In [144]:
pr_sum # 9,2000

0.20644320773582925

In [76]:
pred01 = pred(preds_3_0,0.5) # 7,3000
pred01.value_counts()

0    23620
1      226
dtype: int64

In [77]:
model_for_test

NameError: name 'model_for_test' is not defined

In [15]:
# model3
watchlist = [(dataset3,'train')]
model_3 = xgb.train(params,dataset3,num_boost_round = 2000,evals = watchlist,
                    early_stopping_rounds = 100)

feature_score_3 = model_3.get_fscore()
feature_score_3 = sorted(feature_score_3.items(), key=lambda x:x[1],reverse=True)

[0]	train-logloss:0.684618
Will train until train-logloss hasn't improved in 100 rounds.
[1]	train-logloss:0.676263
[2]	train-logloss:0.668067
[3]	train-logloss:0.660042
[4]	train-logloss:0.652165
[5]	train-logloss:0.644432
[6]	train-logloss:0.636847
[7]	train-logloss:0.629399
[8]	train-logloss:0.622098
[9]	train-logloss:0.614926
[10]	train-logloss:0.60789
[11]	train-logloss:0.600984
[12]	train-logloss:0.594203
[13]	train-logloss:0.587543
[14]	train-logloss:0.580994
[15]	train-logloss:0.574576
[16]	train-logloss:0.56826
[17]	train-logloss:0.562054
[18]	train-logloss:0.55596
[19]	train-logloss:0.549954
[20]	train-logloss:0.544071
[21]	train-logloss:0.53833
[22]	train-logloss:0.532641
[23]	train-logloss:0.527038
[24]	train-logloss:0.52153
[25]	train-logloss:0.516121
[26]	train-logloss:0.510792
[27]	train-logloss:0.505558
[28]	train-logloss:0.500407
[29]	train-logloss:0.495345
[30]	train-logloss:0.490366
[31]	train-logloss:0.485462
[32]	train-logloss:0.48064
[33]	train-logloss:0.475891
[3

[286]	train-logloss:0.140693
[287]	train-logloss:0.140473
[288]	train-logloss:0.140263
[289]	train-logloss:0.140055
[290]	train-logloss:0.139849
[291]	train-logloss:0.139638
[292]	train-logloss:0.139433
[293]	train-logloss:0.13923
[294]	train-logloss:0.139025
[295]	train-logloss:0.138823
[296]	train-logloss:0.138623
[297]	train-logloss:0.138423
[298]	train-logloss:0.13824
[299]	train-logloss:0.138051
[300]	train-logloss:0.137872
[301]	train-logloss:0.137678
[302]	train-logloss:0.137493
[303]	train-logloss:0.137303
[304]	train-logloss:0.137123
[305]	train-logloss:0.136945
[306]	train-logloss:0.136764
[307]	train-logloss:0.136591
[308]	train-logloss:0.13642
[309]	train-logloss:0.136257
[310]	train-logloss:0.136089
[311]	train-logloss:0.135919
[312]	train-logloss:0.135747
[313]	train-logloss:0.135581
[314]	train-logloss:0.135418
[315]	train-logloss:0.135257
[316]	train-logloss:0.135088
[317]	train-logloss:0.134916
[318]	train-logloss:0.134761
[319]	train-logloss:0.134606
[320]	train-loglo

[570]	train-logloss:0.115958
[571]	train-logloss:0.115928
[572]	train-logloss:0.115905
[573]	train-logloss:0.115864
[574]	train-logloss:0.11584
[575]	train-logloss:0.115793
[576]	train-logloss:0.115764
[577]	train-logloss:0.115732
[578]	train-logloss:0.115702
[579]	train-logloss:0.115659
[580]	train-logloss:0.11563
[581]	train-logloss:0.115585
[582]	train-logloss:0.115557
[583]	train-logloss:0.115527
[584]	train-logloss:0.115491
[585]	train-logloss:0.115456
[586]	train-logloss:0.115424
[587]	train-logloss:0.115396
[588]	train-logloss:0.115366
[589]	train-logloss:0.115333
[590]	train-logloss:0.1153
[591]	train-logloss:0.115266
[592]	train-logloss:0.115237
[593]	train-logloss:0.115206
[594]	train-logloss:0.115169
[595]	train-logloss:0.115141
[596]	train-logloss:0.115108
[597]	train-logloss:0.115071
[598]	train-logloss:0.115039
[599]	train-logloss:0.115012
[600]	train-logloss:0.114977
[601]	train-logloss:0.114951
[602]	train-logloss:0.114917
[603]	train-logloss:0.114885
[604]	train-loglos

[854]	train-logloss:0.109182
[855]	train-logloss:0.109154
[856]	train-logloss:0.109132
[857]	train-logloss:0.109123
[858]	train-logloss:0.109085
[859]	train-logloss:0.10907
[860]	train-logloss:0.109054
[861]	train-logloss:0.109022
[862]	train-logloss:0.109002
[863]	train-logloss:0.108983
[864]	train-logloss:0.108976
[865]	train-logloss:0.108937
[866]	train-logloss:0.108927
[867]	train-logloss:0.108897
[868]	train-logloss:0.108873
[869]	train-logloss:0.108846
[870]	train-logloss:0.108802
[871]	train-logloss:0.108767
[872]	train-logloss:0.108756
[873]	train-logloss:0.108753
[874]	train-logloss:0.108733
[875]	train-logloss:0.108711
[876]	train-logloss:0.108707
[877]	train-logloss:0.108692
[878]	train-logloss:0.108649
[879]	train-logloss:0.10863
[880]	train-logloss:0.108615
[881]	train-logloss:0.108593
[882]	train-logloss:0.10857
[883]	train-logloss:0.108558
[884]	train-logloss:0.108526
[885]	train-logloss:0.108519
[886]	train-logloss:0.108508
[887]	train-logloss:0.10849
[888]	train-loglos

[1134]	train-logloss:0.10387
[1135]	train-logloss:0.103862
[1136]	train-logloss:0.103854
[1137]	train-logloss:0.103844
[1138]	train-logloss:0.103834
[1139]	train-logloss:0.103821
[1140]	train-logloss:0.103805
[1141]	train-logloss:0.103781
[1142]	train-logloss:0.103759
[1143]	train-logloss:0.103731
[1144]	train-logloss:0.103703
[1145]	train-logloss:0.103678
[1146]	train-logloss:0.103669
[1147]	train-logloss:0.103647
[1148]	train-logloss:0.103631
[1149]	train-logloss:0.103622
[1150]	train-logloss:0.103589
[1151]	train-logloss:0.103565
[1152]	train-logloss:0.103531
[1153]	train-logloss:0.103518
[1154]	train-logloss:0.103515
[1155]	train-logloss:0.103478
[1156]	train-logloss:0.103466
[1157]	train-logloss:0.103438
[1158]	train-logloss:0.103417
[1159]	train-logloss:0.103412
[1160]	train-logloss:0.103398
[1161]	train-logloss:0.103386
[1162]	train-logloss:0.103377
[1163]	train-logloss:0.103345
[1164]	train-logloss:0.103329
[1165]	train-logloss:0.103321
[1166]	train-logloss:0.103314
[1167]	trai

[1408]	train-logloss:0.098885
[1409]	train-logloss:0.098868
[1410]	train-logloss:0.098863
[1411]	train-logloss:0.098841
[1412]	train-logloss:0.098828
[1413]	train-logloss:0.098821
[1414]	train-logloss:0.098816
[1415]	train-logloss:0.098809
[1416]	train-logloss:0.098791
[1417]	train-logloss:0.098784
[1418]	train-logloss:0.098768
[1419]	train-logloss:0.098755
[1420]	train-logloss:0.09874
[1421]	train-logloss:0.098728
[1422]	train-logloss:0.098714
[1423]	train-logloss:0.098697
[1424]	train-logloss:0.098684
[1425]	train-logloss:0.098669
[1426]	train-logloss:0.098657
[1427]	train-logloss:0.09865
[1428]	train-logloss:0.098635
[1429]	train-logloss:0.098631
[1430]	train-logloss:0.098618
[1431]	train-logloss:0.098604
[1432]	train-logloss:0.098592
[1433]	train-logloss:0.098579
[1434]	train-logloss:0.098564
[1435]	train-logloss:0.098551
[1436]	train-logloss:0.098515
[1437]	train-logloss:0.098498
[1438]	train-logloss:0.09849
[1439]	train-logloss:0.098483
[1440]	train-logloss:0.098455
[1441]	train-

[1683]	train-logloss:0.094581
[1684]	train-logloss:0.094568
[1685]	train-logloss:0.094565
[1686]	train-logloss:0.094558
[1687]	train-logloss:0.094549
[1688]	train-logloss:0.094535
[1689]	train-logloss:0.094525
[1690]	train-logloss:0.094505
[1691]	train-logloss:0.094499
[1692]	train-logloss:0.094494
[1693]	train-logloss:0.094484
[1694]	train-logloss:0.09446
[1695]	train-logloss:0.094446
[1696]	train-logloss:0.094425
[1697]	train-logloss:0.094419
[1698]	train-logloss:0.094415
[1699]	train-logloss:0.094408
[1700]	train-logloss:0.094396
[1701]	train-logloss:0.094381
[1702]	train-logloss:0.09436
[1703]	train-logloss:0.094339
[1704]	train-logloss:0.09432
[1705]	train-logloss:0.094298
[1706]	train-logloss:0.094274
[1707]	train-logloss:0.094268
[1708]	train-logloss:0.094258
[1709]	train-logloss:0.094224
[1710]	train-logloss:0.094208
[1711]	train-logloss:0.094188
[1712]	train-logloss:0.09417
[1713]	train-logloss:0.094148
[1714]	train-logloss:0.094133
[1715]	train-logloss:0.094118
[1716]	train-l

[1958]	train-logloss:0.09054
[1959]	train-logloss:0.09051
[1960]	train-logloss:0.090495
[1961]	train-logloss:0.090492
[1962]	train-logloss:0.090485
[1963]	train-logloss:0.090477
[1964]	train-logloss:0.090457
[1965]	train-logloss:0.090441
[1966]	train-logloss:0.090408
[1967]	train-logloss:0.090393
[1968]	train-logloss:0.090379
[1969]	train-logloss:0.090372
[1970]	train-logloss:0.090362
[1971]	train-logloss:0.090358
[1972]	train-logloss:0.090349
[1973]	train-logloss:0.090336
[1974]	train-logloss:0.090332
[1975]	train-logloss:0.090321
[1976]	train-logloss:0.090306
[1977]	train-logloss:0.090293
[1978]	train-logloss:0.090287
[1979]	train-logloss:0.090267
[1980]	train-logloss:0.090247
[1981]	train-logloss:0.090236
[1982]	train-logloss:0.090221
[1983]	train-logloss:0.090205
[1984]	train-logloss:0.090174
[1985]	train-logloss:0.090162
[1986]	train-logloss:0.090158
[1987]	train-logloss:0.090139
[1988]	train-logloss:0.090137
[1989]	train-logloss:0.090134
[1990]	train-logloss:0.09012
[1991]	train-

In [16]:
feature_score_3

[('room', 46582),
 ('wuli_totalrooms', 38579),
 ('glon', 37529),
 ('glat', 36473),
 ('ordadvanceday', 34211),
 ('zifangxing_totalrooms', 31099),
 ('order_minute', 30050),
 ('order_hour', 24090),
 ('supplierid', 22976),
 ('city', 20150),
 ('zone', 15907),
 ('day_stay', 10532),
 ('countryid', 9134),
 ('hotelstar', 7146),
 ('hotelbelongto', 2242),
 ('supplierchannel', 2216),
 ('isebookinghtl', 1814),
 ('isvendor', 1277),
 ('isholdroom', 1052)]

In [17]:
preds_3 = model_3.predict(dataset3)

In [18]:
pr_sum,precision,recall = pr(y901914,preds_3)

In [19]:
pr_sum # 9,2000

0.40840762931164559

In [ ]:
test结果

In [ ]:
test_A

In [156]:
y_pred = model_3.predict(dataset4)

In [ ]:
test_B

In [20]:
y_pred = model_3.predict(dataset5)

In [21]:
pred01 = pred(y_pred,0.5) # 9,2000
pred01.value_counts()

0    10945
1       91
dtype: int64

In [86]:
这里注意必须调整成原来的test_A的顺序，比较保险

SyntaxError: invalid character in identifier (<ipython-input-86-b433273a5137>, line 1)

In [158]:
temp = test_sub = test_A[['orderid','room']]
temp['noroom'] = y_pred

E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
temp = test_sub = test_B[['orderid','room']]
temp['noroom'] = y_pred

E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
test_A_temp = pd.read_csv('dataset/test/ord_testA.csv',encoding = 'GB2312')
test_A_temp = test_A_temp[['orderid','room','arrival']]
test_B_temp = pd.read_csv('dataset/test/ord_testB.csv',encoding = 'GB2312')
test_B_temp = test_B_temp[['orderid','room','arrival']]

In [109]:
temp.head()

,orderid,room,noroom
0,4508328865,5262899,0.014956
1,4508324366,71194186,0.001403
2,4508329857,79110495,0.056967
3,4508318938,64828242,0.046322
4,4508326739,58550581,0.004521


In [110]:
test_A_temp.head()

,orderid,room,arrival
0,4505896718,71340604,2017/10/11
1,4504466510,70578904,2017/9/19
2,4505791849,91096440,2017/9/16
3,4505284465,20487676,2017/9/17
4,4506807614,88163053,2017/9/17


In [160]:
test_sub = pd.merge(test_A_temp,temp[['orderid','noroom']],how = 'left')

In [24]:
test_sub = pd.merge(test_B_temp,temp[['orderid','noroom']],how = 'left')

In [112]:
test_sub.head() # 8,3000

,orderid,room,arrival,noroom
0,4505896718,71340604,2017/10/11,0.077105
1,4504466510,70578904,2017/9/19,0.023503
2,4505791849,91096440,2017/9/16,0.047205
3,4505284465,20487676,2017/9/17,0.016086
4,4506807614,88163053,2017/9/17,0.049044


In [112]:
test_sub.to_csv('xgb_baseline_0715_15day_depth9_2000_minute_new.csv',index = False)

In [162]:
test_sub.to_csv('xgb_baseline_0715_15day_depth9_2000_minute_new_jingwei.csv',index = False)

In [26]:
test_sub.to_csv('xgb_baseline_0715_15day_depth9_2000_minute_new_testB.csv',index = False)